In [58]:
import pandas as pd
from transformers import pipeline
import time

## Reading all data

In [2]:
data_or = pd.read_csv('/home/luis-carlos/Documents/experiments/nlp/data/whole_synthetic_data.csv')

In [3]:
data_or.head()

,user_input,language,type,Categories
0,このラップトップは、非常に軽量で持ち運びが簡単です。バッテリーの持ちも良く、一日中外出先で使...,jp,review,"Portability, Battery Life"
1,デザインが洗練されていて、キーボードの打ち心地が快適です。ただし、ファンの音が少しうるさいです。,jp,review,"Design, Keyboard, Noise"
2,処理速度が速く、ビジネス用途にも最適です。ExcelやPowerPointの動作がスムーズで...,jp,review,"Performance, Business Use, Applications"
3,ディスプレイが非常に鮮明で、映画を見るのが楽しいです。ただし、スピーカーの音質は改善が必要です。,jp,review,"Display, Entertainment, Sound"
4,ゲーミング用に購入しましたが、重いゲームも問題なく動作します。冷却性能も優れています。,jp,review,"Gaming, Cooling, Performance"


In [24]:
all_categories = data_or['Categories'].apply(lambda x: x.split(',')).tolist()
all_categories = [item for sublist in all_categories for item in sublist]
all_categories = [text.lstrip() for text in all_categories]

In [21]:
all_categories_unique = sorted(list(set(all_categories)))

## Reading main categories

In [37]:
path = '/home/luis-carlos/Documents/experiments/nlp/data/whole_synthetic_data_main_categories.csv'

main_categories_df = pd.read_csv(path)
main_categories_df.head()

,Main Category,Subcategories
0,Gaming & Graphics-Intensive Tasks,3D Modeling
1,Accessories & Power,Accessories
2,AI & Smart Features,AI Accuracy
3,Software & Applications,Applications
4,Keyboard & Typing,Backlighting


## Starting classification
### Zero shot

In [27]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [26]:
main_categories = ['Accessories & Power', 'AI & Smart Features', 'Battery & Charging', 'Boot Time & Speed', 'Build Quality & Design', 'Connectivity', 'Cooling & Thermal Management', 'Customer Support & Warranty', 'Display', 'Ergonomics & User Comfort', 'Gaming & Graphics-Intensive Tasks', 'Keyboard & Typing', 'Performance', 'Portability', 'Return & Policies', 'Software & Applications', 'Sound & Audio', 'Storage', 'Value & Pricing', 'Video & Camera']

In [73]:
i = 58
text = data_or.loc[i, 'user_input']
text

'What is the return policy if I am not satisfied with the laptop?'

In [74]:
a = data_or.loc[i, 'Categories']
a = a.split(', ')
a

['Customer Service', 'Return Policy']

In [75]:
t0 = time.time()
prediction = classifier(text, main_categories)
t1 = time.time() - t0
print(f'Execution time: {t1} seconds')

Execution time: 95.97047400474548 seconds


In [76]:
print(prediction['sequence']) 
print(prediction['labels']) 
print(prediction['scores'])

What is the return policy if I am not satisfied with the laptop?
['Return & Policies', 'Connectivity', 'Value & Pricing', 'Portability', 'Performance', 'Display', 'Build Quality & Design', 'Customer Support & Warranty', 'Ergonomics & User Comfort', 'AI & Smart Features', 'Sound & Audio', 'Cooling & Thermal Management', 'Gaming & Graphics-Intensive Tasks', 'Boot Time & Speed', 'Accessories & Power', 'Video & Camera', 'Battery & Charging', 'Keyboard & Typing', 'Software & Applications', 'Storage']
[0.4142730236053467, 0.06452030688524246, 0.0635870173573494, 0.0540880411863327, 0.04477532207965851, 0.04287511110305786, 0.04101238399744034, 0.039262160658836365, 0.0320218950510025, 0.026884915307164192, 0.025491097941994667, 0.025176262483000755, 0.02346387319266796, 0.019608816131949425, 0.017386246472597122, 0.016840478405356407, 0.014632100239396095, 0.012332022190093994, 0.011231222189962864, 0.010537669062614441]


In [77]:
main_categories_df[main_categories_df['Subcategories'].isin(a)]

,Main Category,Subcategories
19,Customer Support & Warranty,Customer Service
44,Return & Policies,Return Policy
